# Chapitre 1

L'idée avec ce Chapitre c'est de pouvoir explorer le dataset dans sa globalité, essayer d'étudier les différentes corrélations entre les données et de pouvoir trouver des premiers insights ou patterns qui nous serviront par la suite pour le traitement de ces données

1. Data Exploration
2. Data Interpretation
3. Data Quality Problems
4. Data Cleaning
5. Text Processing
6. Data Transformation
7. Manifold Learning
8. Visualisation & Conclusions

In [86]:
import pandas as pd 
import polars as pl

In [87]:
"""Chargement du dataset en mode snapshot 
La data choisie est le 20-10-2025, certaines données peuvent varier mais pour l'instant on part du principe que ce dataset est représentatif
pour les travaux à venir"""

#Chargement du snapshot va se faire avec pandas mais à terme on va migrer sur du polars ...  ,
# plus rapide et avec du multithreading mais on va devoir faire du casting vers pandas pour les librairies comme scikit ou seaborn ... 

SNAPSHOT = "snapshot_dataset_20_10_2025.csv"
CHEMIN_FICHIER = f"./data/{SNAPSHOT}"  

DATASET = pd.read_csv(CHEMIN_FICHIER)
print(DATASET.shape)
print(DATASET.dtypes)

(28180, 18)
Timestamp                                                                                                                                                                                                                                object
How old are you?                                                                                                                                                                                                                         object
What industry do you work in?                                                                                                                                                                                                            object
Job title                                                                                                                                                                                                                                object
If your job title needs addi

Affichage des informations générales sur le dataset

In [88]:
print(DATASET.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28180 entries, 0 to 28179
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             28180 non-null  object 
 1   How old are you?                                                                                             

*** Remarque *** : Ici on remarque que nous avons une colonne numérique au premier abord qui est : How much additional monetary compensation do you have , donc les bonus etc est la seule colonne numérique directement, cependant le annual salary n'est pas en float ... faudra faire le casting.

Cette colonne inclut pas le remboursement du titre de transport, tickets resto etc ... 


## Renommage des colonnes
Pour commencer la visualisation et l'exploration plus détaillée, l'idée c'est de renommer les colonnes avant pour un meilleur confort visuel

In [89]:
df = DATASET
#Supposition : TODO autres_devises devrait être probablement fusionné avec devise, car c'est juste un formulaire et le choix se portait sur les plus connues
# TODO : concaténer état et pays si c'est USA , ville pas forcèment.... 
nouvelles_colonnes = ["timestamp","tranche_age","industrie_travail","intitule_poste","intitule_poste_contexte_supp","salaire_annuel","bonus",
"devise","autre_devise","contexte_revenus","pays_travail","etat_travail","ville_travail","experience_generale","experience_domaine",
"diplome","genre","race"]

def renommer_colonnes(df:pd.DataFrame, nouvelles_colonnes : list) -> pd.DataFrame : 
    dictionnaire_colonnes = {}
    for anc_col , nouv_col in zip(df.columns,nouvelles_colonnes):
        dictionnaire_colonnes[anc_col]=nouv_col
    df_nouveau = df.rename(columns=dictionnaire_colonnes)
    return df_nouveau

df_colonnes = renommer_colonnes(df,nouvelles_colonnes)
print(df_colonnes.head())


            timestamp tranche_age              industrie_travail  \
0  4/27/2021 11:02:10       25-34   Education (Higher Education)   
1  4/27/2021 11:02:22       25-34              Computing or Tech   
2  4/27/2021 11:02:38       25-34  Accounting, Banking & Finance   
3  4/27/2021 11:02:41       25-34                     Nonprofits   
4  4/27/2021 11:02:42       25-34  Accounting, Banking & Finance   

                             intitule_poste intitule_poste_contexte_supp  \
0        Research and Instruction Librarian                          NaN   
1  Change & Internal Communications Manager                          NaN   
2                      Marketing Specialist                          NaN   
3                           Program Manager                          NaN   
4                        Accounting Manager                          NaN   

  salaire_annuel   bonus devise autre_devise contexte_revenus    pays_travail  \
0         55,000     0.0    USD          NaN         

In [90]:
def comparaison_colonnes(df:pd.DataFrame,col1:str,col2:str):
    """Prendre deux colonnes et essayer de comparer les résultats """
    for i , row in df.iterrows():
        if str(row[col2])!="nan":
            print(f"indice {i} : {row[col1]} {row[col2]}\n\n")


def get_pourcentage_valeurs_nulles(df: pd.DataFrame , colonne : str ) -> float : 
    """prend une colonne en particulier et retourne le pourcentage de valeurs nulles contenant"""
    return (df[colonne].isnull().sum()/df.shape[0])*100


# comparaison_colonnes(df_colonnes,"devise","autre_devise")

seuil_valeurs_nulles = 10 #10% sont nulles 
for colonne in df_colonnes.columns : 
    if get_pourcentage_valeurs_nulles(df_colonnes,colonne) >= 10 : 
        print(f"colonne {colonne} est nulle à {get_pourcentage_valeurs_nulles(df_colonnes,str(colonne))}")



colonne intitule_poste_contexte_supp est nulle à 74.15188076650107
colonne bonus est nulle à 26.103619588360537
colonne autre_devise est nulle à 99.22995031937545
colonne contexte_revenus est nulle à 89.17672107877928
colonne etat_travail est nulle à 17.94180269694819


Observations : 

. colonne autre devise vide à 99% => envisager factorisation

. colonne devise pas vide mais cas où nous avons des others, cas autres devises obligatoire

. colonne bonus 26% nulles mais peut être pertinente ...

. etat_travail nulle à 18% indique que la plupart des travailleurs viennent des US .. à voir si c'est vraiment pertinent ou si on factoriserait pas selon le besoin

. contexte revenus nulle à 90% mais à garder certains pouvant justifier leurs revenus avec ça

. intitule poste contexte nulle à 75% .. à garder pour l'instant




In [91]:
comparaison_colonnes(df_colonnes,"devise","autre_devise")

indice 434 : Other INR


indice 603 : Other Peso Argentino


indice 752 : USD $76,302.34


indice 766 : USD My bonus is based on performance up to 10% of salary


indice 776 : USD I work for an online state university, managing admissions data. Not direct tech support. 


indice 1001 : USD 0


indice 1311 : Other MYR


indice 1840 : Other CHF


indice 1915 : USD KWD


indice 1924 : Other NOK


indice 2048 : USD Na 


indice 2054 : Other NOK


indice 2473 : Other USD


indice 2639 : Other BR$


indice 2980 : Other SEK


indice 3124 : USD Base plus Commission 


indice 3145 : CAD canadian


indice 3162 : Other Dkk


indice 3261 : Other EUR


indice 3605 : USD COP


indice 3937 : Other NOK


indice 4264 : Other TTD


indice 4499 : Other Indian rupees


indice 4780 : Other BRL (R$)


indice 4971 : Other Mexican pesos


indice 5146 : Other CZK


indice 5306 : Other CZK


indice 5396 : Other GBP


indice 5729 : Other NOK


indice 5871 : Other GBP


indice 6128 : Other DKK


indice 6765 : Oth

In [92]:
import matplotlib.pyplot as plt
from typing import Literal , List , Optional
import seaborn as sea

In [93]:
def visualisation_distribution_donnees(df : pd.DataFrame , colonne : str , type_graphique:Literal["bar","pie","hist"]) : 
    """visualiser selon le type de graphique et la colonne qu'on choisit"""
    figsize = (10,8)
    plt.figure(figsize=figsize)
    if type_graphique == "bar" : 
        sea.barplot(x=df[colonne].value_counts().index,y=df[colonne].value_counts().values)
        plt.title(f'Distribution de {colonne}')
        plt.xlabel(f'{colonne}')
        plt.ylabel('Count')
        plt.show()  
    if type_graphique == "pie":
        plt.pie(df[colonne].value_counts().values, labels=df[colonne].value_counts().index, autopct='%1.1f%%', startangle=90,labeldistance=1.2)
        plt.title(f'Distribution de {colonne}')
        plt.axis('equal') 
        plt.show()
    if type_graphique == "hist":
        sea.histplot(data=df,x=colonne,bins=300,log_scale=False)
        plt.ticklabel_format(style='plain', axis='x')  #désactive le exposant6 et affiche le chiffre dans son entièreté.
        plt.show()


def verifier_colonne_numerique(df:pd.DataFrame,colonne:str) -> None : 
    """Cette fonction viseprincipalement à faire en sorte de vérifier certaines colonnes numériques parfois problématiques"""
    print(df[colonne].describe())  
    print("Max:", df[colonne].max())
    print("Min:", df[colonne].min())


# df_colonnes["bonus"] = df_colonnes["bonus"].dropna()
moyenne = df_colonnes["bonus"].mean()
ecart_type = df_colonnes["bonus"].std()
print(df_colonnes['bonus'].dropna().sort_values())#on affiche le mode qui pour l'instant est à 0 .. ici 
# print("moyenne", moyenne ,"ecart_type",ecart_type,"mode")

# verifier_colonne_numerique(df_colonnes , "bonus")




0                0.0
16126            0.0
16128            0.0
16129            0.0
7170             0.0
            ...     
18322      1400000.0
26935      1500000.0
26980      2000000.0
28100      2000000.0
11454    120000000.0
Name: bonus, Length: 20824, dtype: float64


Observation : on remarque que la valeur max en temps normal pour la colonne bonus la valeur max est à 12 millions, 
là où la moyenne est beaucoup plus basse, dans ce cas on va décider de garder que les valeurs 3 sigmas au délà de la moyenne au délà on affiche la colonne pour investiguer 
Il s'agit d'un problème de la monnaie avec lesquels les gens sont payés ... faudra faire un comparatif sur la distribution de données apparatenant à la même catégorie devise ...

# 2. Data interpretation

Objectif de cette partie est de résumer ce qu'on a pu voir lors de la section précédente, expliquer chaque feature(colonne après renommage), sa typologie, son utilité analytique et les difficultés potentielles qu'on va rencontrer sur cette colonne ( au moment du snapshot)


1- colonne timestamp : le type est une chaine de caractère qui correspond à la date de soumission du questionnaire probablement ou dernière modification, pour l'instant pas d'utilité mais ça peut être intéressant pour voir notamment l'évolution des salaires avant et post covid notamment.

2- tranche_age : catégorie ordonnée, avec 0% de valeurs nulles et la catégorie 25-34 qui domine avec 45%, peut être corrélée au niveau d'expérience et éventuellement au salaire mais pour le traiter faudra trouver un moyen d'encoder les données de cette colonne

3- industrie_travail : ça a l'air d'être des catégories (une liste déroulante au moment de la sélection) avec moins de 1% de valeurs nulles, elle représente le secteur d'activité au sens plus global 
comme c'est une liste déroulante on aura pas de soucis avec les variations d'écriture ( tech, technology, it...) mais pourrait nécessiter soit un regroupement et le traitement des valeurs manquantes.

4- intitule_poste : ici on a une donnée textuelle, qui représente le poste de la personne, seules 2 valeurs sont manquantes qu'on peut ignorer mais on a la moitié des valeurs qui sont des valeurs distinctes, donc le champ du formulaire était probablement un input texte, son utilité résidera dans l'analyse des salaires en donnant plus d'informations qu'avant => la difficulté sera donc le regroupement de certains postes (software engineer, developer ou chef de projet et chef projet par exemple) traiter le bruit lexical et éventuellement encoder l'information avec des embeddings ou du tfidf

5-intitule_poste_contexte_supp : zone de texte libre, qui est nulle à 74%, mais peut avoir une utilité sur la distinction de certains postes similaires ou le regroupement de certains postes jugés éloignés au début, l'utilité est faible en raison de la complexité du traitement et le nombre de valeurs manquantes mais on écarte pas la piste pour l'instant

6- salaire_annuel : stocké en texte mais c'est une donnée numérique, pas de valeurs nulles et l'utilité sera de mener les études sur le profil de certaines personnes, prédictions de salaires ..., la difficulté sera donc de traiter cette donnée en la numérisant d'abord, et traiter les outliers et éventuellement, le cas multi devise à traiter et à unifier (ce qui n'est pas évident comme on le verra après)

7- Bonus : comme le salaire_annuel, ici on a une compensation additionnel en plus du salaire_annuel, peut être ajoutée au salaire_total, mais présente 26% de valeurs nulles qu'on va imputer à 0 et le cas multidevises à gèrer

8- devise : variable catégorielle représentant la devise du salaire_annuel, bonus etc, une variable indispensable dans la normalisation des salaires, probablement une liste déroulante grâce à la valeur other qui nous donne un bon indicateur sur ça .

9-autre_devise : ici 99% valeurs sont nulles on pourrait fusionner les valeurs intéressantes (car c'est du texte) dans la colonne précédente en supprimant le other à la place, mais ça nécessitera de traiter ce texte (valeurs numériques parfois des lignes entières avec des personnes qui se sont trompées de champ (index 776 , ou 1001))

10-contexte_revenus: :explication du salaire, peut apporter de l'information supplémentaire qu'on peut modèliser (une nouvelle colonne travail_supp : avec des champs qu'on va contrôler comme freelance, temps partiel) avec du nlp, regex , ou éventuellement un llm 

11- pays_travail : variable explicative du salaire, analyser la répartition par zone géographique mais soucis (U.S , USA , United States) qui peut être pénible à gèrer et qui nécessiterait des regroupement. 

12- etat_travail : si pays c'est USA => etat_travail pas null, utilité si on veut analyser que le marché américain, car hors usa elle est à null, et on a 137 valeurs distinctes ce qui veut dire qu'on a du bruit lexicale, erreurs de frappe ou des errerus de saisie dans le champ car 50 états possibles en théorie.

13- ville_travail : utilité pas très intéressante car zone géographique trop précise et on recense parfois des villes dont le nom est similaire dans deux pays différents ce qui rend cette colonne inexploitable 

14 - experience_generale : tout comme âge, colonne catégorique ordinale stockée en texte, qui peut être cruciale pour comprendre les salaires, faudra encoder cette colonne selon la typologie de cette variable 

15 - experience_domaine : idem que la précédente variable, utile pour clusteriser les niveaux de séniorité des différentes personnes.

16-diplome : plus haut niveau d'éducation, utile pour trouver lien diplome-salaire, mais comme c'est des catégories ça reste plus simple à traiter, on peut traiter ça comme une catégorie ordonnée également, et on a très peu de valeurs nulles (soit pas de diplôme ou à creuser)

17- genre : catégorie (5 valeurs distinctes) avec quelques valeurs nulles, peut servir pour mener une analyse démographique 

18- race : 52 catégories environ, qui représente les identités ethniques déclarées, pour l'instant on la garde comme un attribut descriptif mais sans plus ... 


### Cas d'usages problématiques à trouver à partir de ces données 

Parmi les problématiques intéressantes et les études qu'on pourrait mener sur ce dataset on trouve par exemple : 

- La régression : ici on pourrait chercher à prédire les salaires annuels, à partir du contexte démographique , géographique et parcours professionnels (poste actuels , diplome...) des différentes personnes  identifier les différentes corrélations entre les données.

- Classification : ici on pourrait chercher à prédire notamment des classes, comme par exemple faire de la classification sur le niveau de séniorité d'une personne,  mais cela reviendrait à labelliser le dataset et définir ce qu'est un niveau de séniorité (combien d'années d'expérience nous faut il par domaine pour devenir senior ou pas), ce qui pourrait être intéressant selon le parcours académique, poste actuel et expérience globale et même le pays de voir comment les gens deviennent séniors et de donner ces stats par exemple à une entreprise pour savoir quel profil embaucher ou quel profil a plus de chance de devenir senior rapidement ( c'est assez complexe car senior ne veutr pas toujours dire plus compétent qu'un junior mais effectivement ça permet d'avoir cet oeil sur ce genre de statistiques, avec aussi éventuellement un enrichissement de données nécessaire si les données actuelles ne suffisent pas à déterminer cette problèmatique )

- Clustering : ici le but sera de trouver les structures cachées dans les données, notamment pour la détection d'anomalies sur les salaires/Bonus incohérents ou avec devise erronée, faire du clustering également pour faire sortir une segmentation salairiales mondiale ou ce qu'on appelle le CSP ce qui pourrait être intéressant de voir ce qui détermine par exemple un CSP A aux usa (probablement les profils tech), et la classe moyenne en europe par exemple ... 
Dernière problématique qui pourrait être intéressant ce serait de se situer entre l'industrie de travail et l'intitulé de poste , se positionner entre les deux et de nous dire, on aimerait regrouper tous les métiers liés à la data, les métiers liés au développement logiciel , autres au ingénieurs systèmes et réseau ..., pour réduire notamment le bruit lexical inhérents aux propriétés de la colonne job_title car ça reste du texte libre et donc assez difficile à classifier par la suite

# 3. Problématiques détectées sur la qualité des données 

On recense deux macro catégories dans notre dataset : 

données numériques : ce qu'il y a en haut mais en mieux 

données textuelles : résumer ce qu'il y a en haut avec solutions également 

## 4. Data cleaning :

Partie 1 :  prétraitement des données tabulaires 

In [94]:
def standardiser_et_uniformiser_devise(df: pd.DataFrame) -> pd.DataFrame:
    df_sortie = df.copy()
    df_sortie['devise'] = df_sortie['devise'].astype(str).str.lower()
    df_sortie['autre_devise'] = df_sortie['autre_devise'].astype(str).str.lower()

    """récupéré à partir des données que j'ai recueillies dans la colonne autre devise puis on a standardisé la currency"""
    dictionnaire_devise = {
        'usd': 'USD', 
        'us dollar': 'USD', 
        'american dollars': 'USD',
        'gbp': 'GBP', 
        'eur': 'EUR', 
        'euro': 'EUR',
        'cad': 'CAD',
        'aud/nzd': 'AUD', 
        'aud': 'AUD', 
        'australian dollars ': 'AUD', 
        'aud australian ': 'AUD',
        'inr': 'INR', 'indian rupees': 'INR', 'inr (indian rupee)': 'INR',
        'peso argentino': 'ARS', 'ars': 'ARS', 'argentinian peso (ars)': 'ARS', 'argentine peso': 'ARS',
        'chf': 'CHF',
        'myr': 'MYR',
        'zar': 'ZAR',
        'sek': 'SEK',
        'hkd': 'HKD',
        'nok': 'NOK', 'norwegian kroner (nok)': 'NOK',
        'br$': 'BRL', 'brl (r$)': 'BRL', 'brl': 'BRL',
        'dkk': 'DKK', 'dkk ': 'DKK', 'danish kroner': 'DKK',
        'ttd': 'TTD',
        'mexican pesos': 'MXN', 'mxn': 'MXN',
        'czk': 'CZK', 'czech crowns': 'CZK',
        'bdt': 'BDT',
        'php': 'PHP', 'philippine peso': 'PHP', 'philippine peso (php)': 'PHP', 'philippine pesos': 'PHP',
        'pln (polish zloty)': 'PLN', 'pln': 'PLN', 'polish złoty': 'PLN', 'pln (zwoty)': 'PLN',
        'try': 'TRY',
        'cny': 'CNY', 'rmb (chinese yuan)': 'CNY', 'china rmb': 'CNY',
        'ils/nis': 'ILS', 'nis (new israeli shekel)': 'ILS', 'ils (shekel)': 'ILS', 'ils': 'ILS', 'israeli shekels': 'ILS',
        'jpy': 'JPY',
        'taiwanese dollars': 'TWD', 'ntd': 'TWD',
        'sgd': 'SGD', 'sgd ': 'SGD', 'singapore dollara': 'SGD',
        'krw (korean won)': 'KRW', 'krw': 'KRW', 'korean won ': 'KRW',
        'thb': 'THB', 'thai  baht': 'THB', 'thai baht ': 'THB', 'thai baht': 'THB',
        'idr ': 'IDR', 'idr': 'IDR',
        'nzd': 'NZD',
        'lkr': 'LKR',
        'croatian kuna': 'HRK',
        'rupees': 'INR', 'rupees ': 'INR', 
        'ngn': 'NGN',
        'cop': 'COP', 'pesos colombianos': 'COP',
        'tzs': 'TZS',
        'sar': 'SAR',
        'rm': 'MYR', 
        'equity': 'USD',
        'other': 'Other'
    }

    df_sortie['devise'] = df_sortie['devise'].map(dictionnaire_devise)
    filtre = (df_sortie['devise'] == 'Other') & (df_sortie['autre_devise'] != '') & (df_sortie['autre_devise'] != 'nan')
    df_sortie.loc[filtre, 'devise'] = df_sortie.loc[filtre, 'autre_devise'].map(dictionnaire_devise)
    df_sortie = df_sortie[df_sortie['devise'] != 'Other']
    df_sortie = df_sortie.dropna(subset=['devise'])
    return df_sortie


def conversion_salaire_bonus_to_usd(df:pd.DataFrame,logique_gestion_erreur:Literal['coerce','raise','ignore']='coerce')->pd.DataFrame :
    #par défaut coerce => transforme en nan directement, on peut faire directement avec raise mais généré des erreurs 
    fichier_conversion = "./data/conversion_to_dollars.csv"
    conversion_dict = {}
    with open(fichier_conversion, mode="r", encoding="utf-8") as f:
        lignes = f.readlines()
        for ligne in lignes[1:]:
            tuple = ligne.strip().split(',')
            if len(tuple) == 2:
                devise, taux = tuple[0], float(tuple[1])
                conversion_dict[devise] = taux

    df_sortie = df.copy()    
    df_sortie['salaire_annuel'] = pd.to_numeric(
        df_sortie['salaire_annuel'].astype(str).str.replace(',', '').str.replace(' ', ''),
        errors=logique_gestion_erreur
    )
    df_sortie['bonus'] = pd.to_numeric(
        df_sortie['bonus'].astype(str).str.replace(',', '').str.replace(' ', ''),
        errors=logique_gestion_erreur
    ).fillna(0) 
    df_sortie['taux'] = df_sortie['devise'].map(conversion_dict)
    df_sortie['salaire_annuel_usd'] = df_sortie['salaire_annuel'] * df_sortie['taux']
    df_sortie['bonus_usd'] = df_sortie['bonus'] * df_sortie['taux']    
    df_sortie = df_sortie.drop(columns=['taux'])
    return df_sortie

def seuillage_salaire_minimum(df,seuil_bas_en_dollar = 100,seuil_haut_en_dollar=10000000):
    #à ajuster le seuil car j'ai l'impression que ce qui se situe en dessous de cette tranche n'est pas pertinent du tout
    return df[(df["salaire_annuel_usd"] >= seuil_bas_en_dollar) & (df["salaire_annuel_usd"] <= seuil_haut_en_dollar)]


def nettoyer_donnes_numerique(df : pd.DataFrame) -> pd.DataFrame : 
    df_devise = standardiser_et_uniformiser_devise(df)
    df_conversion =  conversion_salaire_bonus_to_usd(df_devise)
    return seuillage_salaire_minimum(df_conversion)

df_num_nettoye = nettoyer_donnes_numerique(df_colonnes)
print("Shape :", df_num_nettoye.shape)
for i , row in df_num_nettoye.iterrows():
    print(f"ligne {i}  salaire_annuel_usd : {row['salaire_annuel_usd']} ,bonus_usd : {row['bonus_usd']} , BONUS : {row['bonus']} , salaire_annuel : {row['salaire_annuel']} , {row['devise']}")

Shape : (28062, 20)
ligne 0  salaire_annuel_usd : 55000.0 ,bonus_usd : 0.0 , BONUS : 0.0 , salaire_annuel : 55000 , USD
ligne 1  salaire_annuel_usd : 69342.0 ,bonus_usd : 5080.0 , BONUS : 4000.0 , salaire_annuel : 54600 , GBP
ligne 2  salaire_annuel_usd : 34000.0 ,bonus_usd : 0.0 , BONUS : 0.0 , salaire_annuel : 34000 , USD
ligne 3  salaire_annuel_usd : 62000.0 ,bonus_usd : 3000.0 , BONUS : 3000.0 , salaire_annuel : 62000 , USD
ligne 4  salaire_annuel_usd : 60000.0 ,bonus_usd : 7000.0 , BONUS : 7000.0 , salaire_annuel : 60000 , USD
ligne 5  salaire_annuel_usd : 62000.0 ,bonus_usd : 0.0 , BONUS : 0.0 , salaire_annuel : 62000 , USD
ligne 6  salaire_annuel_usd : 33000.0 ,bonus_usd : 2000.0 , BONUS : 2000.0 , salaire_annuel : 33000 , USD
ligne 7  salaire_annuel_usd : 50000.0 ,bonus_usd : 0.0 , BONUS : 0.0 , salaire_annuel : 50000 , USD
ligne 8  salaire_annuel_usd : 112000.0 ,bonus_usd : 10000.0 , BONUS : 10000.0 , salaire_annuel : 112000 , USD
ligne 9  salaire_annuel_usd : 45000.0 ,bonus_u

In [ ]:
import pycountry #librairie intéressante pour uniformiser les pays 

#Documentation de la librairie : https://pypi.org/project/pycountry/

def clean_job_title(df):
    """cas intéressant: les étudiants qui se déclarent en tant que tels dans les formulaires, aucun revenu ... 
    décision à prendre => les supprimer car aucun intêret"""
    df = df[~df["intitule_poste"].isnull()]

    termes_aberrants = [
        'student', 'bum', 'unemployed', 'intern', 'volunteer', 'housewife', 'retired',
        'none', 'n/a', 'unknown', 'test', 'demo', 'fake', 'joke',"Thumbs Up guy","vvvv"
    ]

    df['intitule_poste_lower'] = df['intitule_poste'].str.lower()
    df = df[~df['intitule_poste_lower'].str.contains('|'.join(termes_aberrants), na=False)]
    df = df.drop(columns=['intitule_poste_lower'])
    return df

def clean_pays_travail(df):
    """colonne texte à traiter => pays contient 375 valeurs distinctes, ce qui n'est pas normal et on trouve US, USA , us ... """
    df_sortie = df.copy()
    df_sortie['pays_cleaned'] = df_sortie['pays_travail'].astype(str).str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip() #uniformiser aavant le lookup et ajouter .str avec strip pour éviter erreurs 
    df_sortie = df_sortie.reset_index(drop=True)
    
    def _correspondance_pays(nom:str):
        #censée retourner le pays approximé mais si c pas le cas on garde celui de base et on traite manuellement a la fin
        try : 
            pays_lookup = pycountry.countries.lookup(nom) #se base sur une norme pas toujours respectée ( cas Taiwan république de chine)
        except LookupError :
            if nom !='':
                return nom #nom de base
            else : 
                return None
        
    df_sortie["pays_cleaned"] = df_sortie["pays_cleaned"].apply(_correspondance_pays)
    
    #après ça il reste anomalies : unitedstated pas pris en compte, parfois on a des phrases carrèment, qu'on doit traiter 

    return df_sortie

def clean_text_data(df_entree : pd.DataFrame) -> pd.DataFrame: 
    pass


print("avant nettoyage " , df_num_nettoye["pays_travail"].unique()) # 375outputs
df_pays_nettoye = clean_pays_travail(df_num_nettoye)
print(" \napres nettoyage ",df_pays_nettoye['pays_cleaned'].unique())  


avant nettoyage  ['United States' 'United Kingdom' 'US' 'USA' 'Canada' 'United Kingdom '
 'usa' 'UK' 'Scotland ' 'U.S.' 'United States ' 'The Netherlands'
 'Australia ' 'Spain' 'us' 'Usa' 'England' 'United States of America'
 'France' 'United states' 'Scotland' 'USA ' 'United states ' 'Germany'
 'UK ' 'united states' 'Ireland' 'India' 'Australia' 'Uk'
 'United States of America ' 'U.S. ' 'canada' 'Canada ' 'U.S>' 'ISA'
 'Argentina' 'Great Britain ' 'US ' 'United State' 'U.S.A' 'Denmark'
 'U.S.A.' 'America' 'Netherlands' 'netherlands' 'England '
 'united states of america' 'Ireland ' 'Switzerland' 'Netherlands '
 'Bermuda' 'Us' 'The United States' 'United State of America' 'Germany '
 'Malaysia' 'Mexico ' 'United Stated' 'South Africa ' 'Belgium'
 'Northern Ireland' 'u.s.' 'South Africa' 'UNITED STATES' 'united States'
 'Sweden' 'Hong Kong' 'Kuwait' 'Norway' 'Sri lanka' 'Contracts'
 'USA-- Virgin Islands' 'United Statws' 'England/UK' 'U.S'
 "We don't get raises, we get quarterly bonuses